In [1]:
%matplotlib inline

import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import geopandas as gpd
import mplleaflet
import numpy as np
import matplotlib.pyplot as plt

SQL_SOURCE = 'https://htv210.cartodb.com/api/v2/sql'

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

## Read Data

In [2]:
dischargeQuery = '''
SELECT facility_id, count(facility_id)

FROM nyc_discharge_2013
WHERE ccs_diagnosis_description = 'ASTHMA'
GROUP BY facility_id
ORDER BY count DESC
'''
dischargeStream = queryCartoDB(dischargeQuery, format='CSV')
discharge       = pd.read_csv(dischargeStream)
discharge.head()

,facility_id,count
0,1172,1878
1,1169,1786
2,1301,1057
3,1165,920
4,1178,890


In [3]:
facilityQuery = '''
SELECT *

FROM nyc_facilities

WHERE facility_id in (1172, 1169, 1301,1165, 1178)
'''
facilityStream = queryCartoDB(facilityQuery, format='GeoJSON')
facilityData   = json.loads(facilityStream.read())
facilities     = gpd.GeoDataFrame.from_features(facilityData['features'])
facilities.head()

,cartodb_id,cooperator_address,cooperator_address_2,cooperator_city,cooperator_name,cooperator_state,cooperator_zip_code,description,facility_address_1,facility_address_2,...,operator_address_1,operator_address_2,operator_city,operator_name,operator_state,operator_zip_code,ownership_type,regional_office,regional_office_id,short_description
0,624,,,,,New York,NaN,Hospital,1400 Pelham Parkway,,...,125 Worth Street,,New York,New York City Health and Hospital Corporation,New York,10013,Municipality,Metropolitan Area Regional Office - New York City,5,HOSP
1,627,111 East 210th Street,,Bronx,Montefiore Medical Center,New York,10467,Hospital,111 East 210th Street,,...,111 East 210th Street,,Bronx,"Montefiore Health System, Inc",New York,10467,Not for Profit Corporation,Metropolitan Area Regional Office - New York City,5,HOSP
2,628,,,,,New York,NaN,Hospital,234 East 149th Street,,...,125 Worth Street,,New York,New York City Health and Hospital Corporation,New York,10013,Municipality,Metropolitan Area Regional Office - New York City,5,HOSP
3,631,,,,,New York,NaN,Hospital,1650 Grand Concourse,,...,1276 Fulton Avenue,,Bronx,Bronx Lebanon Hospital Center Inc,New York,10456,Not for Profit Corporation,Metropolitan Area Regional Office - New York City,5,HOSP
4,696,,,,,New York,NaN,Hospital,451 Clarkson Avenue,,...,125 Worth Street,,New York,New York City Health and Hospital Corporation,New York,10013,Municipality,Metropolitan Area Regional Office - New York City,5,HOSP


In [4]:
facilities.crs = {'init': 'epsg:4326', 'no_defs': True}
facilities = facilities.to_crs(epsg=2263)

In [5]:
facilities['geometry']

0    POINT (1026645.436320588 251684.2060465284)
1     POINT (1017278.463616514 259890.221163824)
2    POINT (1005228.021716391 237182.1691768789)
3    POINT (1008724.037560453 246639.2619315821)
4    POINT (999760.5456509431 178215.3371727151)
Name: geometry, dtype: object